<center><h1>Discriminatory Practices in Facial Recognition</h1></center>
<center><h4>By Tyler J. Russell, July 2021.</h4></center>

---

# Abstract

Studies have demonstrated for quite some time that there is bias in facial recognition technology and other artificial intelligence models. In this work I measure the accuracy of Microsoft Azure Cognitive Services, Face++, and Amazon Rekognition on a diverse dataset to perform an intersectional benchmark of facial analysis features within facial recognition and detection - specifically, based upon age and gender. I contrast this against the prior Gender Shades project study, concluding that there is still significant evidence of discrimination against those of darker skin tones and females when training facial recognition technology.

# Introduction

Facial recognition error rate for women with darker skin tones is approximately 30.4% higher on average (A1) than that of men with lighter skin tones ([1]). In this investigation, I aim to measure how much this has improved since the Gender Shades project study was conducted in 2018.

*A1*: Calculated by the difference between the average error rate for dark-skinned female and light-skinned male subjects in the dataset, as $\frac{23.8 + 36.0 + 33.1}{3} - \frac{0 + 0 + 1.6}{3} \approx 30.4\%$

[1]: https://sitn.hms.harvard.edu/flash/2020/racial-discrimination-in-face-recognition-technology

# Methodology

## Foreword

This study uses "gender" to refer to "biological sex", not "gender identity", in line with the original Gender Shades project. Note that generally these terms are not interchangeable, and this study has no intention of making them so. In line with ethical concerns, this study does not measure accuracy on anyone below the age of 18.

## Categorizations

To perform an intersectional benchmark for gender classifications and race, we must categorize the dataset. This produces a binary category for gender classification - female, and male, since these are used by the technologies themselves - and a classification for race based on the Fitzpatrick Skin Type metric - those of $I$, $II$, or $III$ phenotypes are considered light-skinned (L), and - those of $IV$, $V$, or $VI$ phenotypes are considered dark-skinned (D). Given that the Gender Shades dataset is not public access, and we do not want to give the models the benefit of having been trained solely with that particular dataset, I will create (and share publicly) a new one that is equally distributed for both included genders and both racial classifications.

I am using the Fitzpatrick Skin Type metric as phenotypical racial analysis is the only scientifically rooted and fair way to classify people by race, and it is used frequently by dermatologists.

Overall, I will measure predicted gender and predicted age from the services' outputs. This will be used to determine the accuracy of identification for each measured class. The accuracy of age will be scored, rather than used for a confusion matrix, determined by its deviation from the truth as a percentage (where higher is worse).

## Variables

I will measure the precision (confidence metric, PPV), error rate (FDR), true-positive rate (TPR), false-positive rate (FPR) and accuracy (ACC) for each service under each classifier.

1. $PPV = \frac{TP}{TP + FP}$
2. $FDR = 1 - PPV$
3. $TPR = \frac{TP}{P}$
4. $FPR = \frac{FP}{N}$
5. $ACC = \frac{TP + TN}{P + N}$

To ensure fair testing, I will use the same model each time for a particular service and will keep the ages of candidates in the dataset consistent.

## Scope

I will collect solely quantitative data using the same dataset for each of three modern neural networking models, which are as follows: Microsoft Azure Cognitive Services, Face++, and Amazon Rekognition (A1).

Due to platform limitations of these services, I can only use a dataset of 4000 images, and the source code for this study may only conduct a maximum of 20 requests per minute.

*A1: IBM ceased research and operation of its facial recognition technology, so despite being included in the prior data, I cannot compare its modern score and have therefore omitted it from this study.*

## Data

I started with the UTKFaces dataset ([4]), as it is very diverse and large enough to trim to my desired sample size whilst still maintaining representation. Furthermore, it is pre-emptively annotated by age, race, and gender - perfect for my criteria. I will omit anyone below the age of 18 from the data due to ethical concerns, and anyone over the age of 80 due to concerns with aged facial features and population norms. This reduced the dataset to 5509 entries.

Following this, I needed to obtain an accurate 4000 face sample that represented each racial phenotype and gender equally, maintaining two distinctive age groups, producing the following sample sizes:

- 2000 of each racial phenotype
- 2000 male (M), 2000 female (F)

Simplified to the following sample groups:

- 1000 per racial phenotype of each gender, creating 4 categories of 1000 faces each in a sample of 4000

For example, a Fitzpatrick Skin Type metric I scoring male would be denoted in the dataset as `LM`.

After having counted the remaining data following the removal of $<18$ and $>80$ year olds, I lacked the requisite number of `DM` individuals. To rectify this, I used the CelebA dataset ([5]) and sorted for 200 more `DM` criteria fitting individuals, and merged the two. Note that these CelebA dataset-merged individuals will not be used in the age accuracy benchmark, as their actual age is unknown (CelebA does not annotate subjects by age). Following this, to trim the dataset further to exactly fit the sample size, I used a random sample to select 1000 from each group.

The dataset variation cleansed and used here has been made public prior to the release of this work. You may access it at reference [6] of this paper.

[4]: https://susanqq.github.io/UTKFace/
[5]: http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html
[6]: https://github.com/Nytelife26/transparency/blob/main/works/personal/2021/nrp/final.tar.gz

## Parsing Data for Results

I will be using the following code to parse image names into their respective age, gender and race, which I will append to the dataset results for comparison.

In [14]:
import re

FILENAME_REGEX = re.compile("^faces/([\dX]{2})_(\d)_(\d)_.+\.jpg$")

def parse_file(fn: str) -> str:
    age, gender, race = FILENAME_REGEX.match(fn).groups()
    try:
        age = int(age)
    except:
        age = -1
    gender = "F" if int(gender) else "M"
    race = "D" if int(race) else "L"
    return {"age": age, "gender": gender, "race": race}

# Collection

## Source

Here is the base code I will be using for my data collection:


```python
import time
from io import open, BufferedReader


class BaseService:
    name: str
    def sdk(self, file: BufferedReader) -> dict: ...


def analyze(services: list[BaseService], files: list[str], wait: int = 0):
    results = {}
    for file in files:
        results[file] = {}
        for service in services:
            results[file][service.name] = service.sdk(open(file, "rb"))
        time.sleep(wait)
    return results
```


This will enable me to define modular services to pass for analysis with a queries-per-second limiter to keep under the ratelimit. The source code for all individual services will be published alongside this paper ([3]).

[3]: https://github.com/Nytelife26/transparency/blob/main/works/personal/2021/nrp

# Results

## Calculations

I will be using the following algorithms to determine the age score and gender-based score for each metric of the intersectional benchmark.

In [13]:
from math import fabs

def age_score(age, desired):
	return fabs(round((age / desired) * 100 - 100, 2))

def gender_score(cat, gender, desired, matrix):
	cat_match = cat.lower() == desired.lower()
	gen_match = gender.lower() == desired.lower()
	
	if cat_match and gen_match:
		matrix["TP"] += 1
	elif not cat_match and gen_match:
		matrix["TN"] += 1
	elif cat_match and not gen_match:
		matrix["FN"] += 1
	elif not cat_match and not gen_match:
		matrix["FP"] += 1
		
	return matrix

Then, I used the following code to attach the metadata from each file to the object itself.

```python
import json

results = json.load(open("results.json", "r"))
for file in results.keys():
    results[file]["meta"] = parse_file(file.split("/")[1])
json.dump(results, open("results.json", "w+"))
```

Finally, to generate confusion matrices for each category, I used the following code.

```python
import json

results = json.load(open("results.json", "r"))

categories = {"M": {}, "F": {},"LM": {}, "LF": {}, "DM": {}, "DF": {}}

def parse_rg(category):
    gender = re.findall("[MF]", category)
    gender = gender[0] if len(gender) else None
    race = re.findall("[DL]", category)
    race = race[0] if len(race) else None
    return gender, race

def scan_data(data, category, service = None):
    new = {}
    gender, race = parse_rg(category)
    for key, val in data.items():
        if (race and val["meta"]["race"] == race) or not race:
            new[key] = val
    return new

services = ["azure", "face++", "rekognition"]
for x in categories.keys():
    gender = parse_rg(x)[0]
    print(gender)
    print(x)
    data = scan_data(results, x)
    for y in services:
        matrix = {"TP": 0, "TN": 0, "FP": 0, "FN": 0}
        for z in data.values():
            gender_score(gender, z[y]["gender"], z["meta"]["gender"], matrix)
        categories[x][y] = matrix
    matrix = {"TP": 0, "TN": 0, "FP": 0, "FN": 0}
    for y in categories[x].values():
        for z in y.keys():
            matrix[z] += y[z]
    categories[x]["total"] = matrix
```

## Final Data

The aforementioned software produces the following table of results. This data in its untabulated format is recorded at `matrices.json`, reference [3].

[3]: https://github.com/Nytelife26/transparency/blob/main/works/personal/2021/nrp

```
       |_____________M_____________|_____________F_____________|____________L_M____________|____________L_F____________|____________D_M____________|____________D_F____________|
       |  TP  |  TN  |  FP  |  FN  |  TP  |  TN  |  FP  |  FN  |  TP  |  TN  |  FP  |  FN  |  TP  |  TN  |  FP  |  FN  |  TP  |  TN  |  FP  |  FN  |  TP  |  TN  |  FP  |  FN  |
 ------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|
 MSFT  | 1943 | 1985 |  15  |  57  | 1985 | 1943 |  57  |  15  |  983 |  989 |  10  |  17  |  989 |  983 |  17  |  10  |  960 |  996 |   5  |  40  |  996 |  960 |  40  |   5  |
 FPP   | 1942 | 1973 |  27  |  58  | 1973 | 1942 |  58  |  27  |  983 |  989 |  10  |  17  |  989 |  983 |  17  |  10  |  959 |  984 |  17  |  41  |  984 |  959 |  41  |  17  |
 AMZN  | 1861 | 1984 |  16  | 139  | 1984 | 1861 | 139  |  16  |  935 |  991 |   8  |  65  |  991 |  935 |  65  |   8  |  926 |  993 |   8  |  74  |  993 |  926 |  74  |   8  |
 ------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|------|
 total | 5746 | 5942 |  58  | 254  | 5942 | 5746 | 254  |  58  | 2901 | 2969 |  28  |  99  | 2969 | 2901 |  99  |  28  | 2845 | 2973 |  30  | 155  | 2973 | 2845 | 155  |  30  |
```


Finally, these are our $PPV$, $FDR$, $TPR$, $FPR$, $ACC$ and age scores (where $T$ is all categories). For $PPV$, $TPR$ and $ACC$, higher is better. For all others, lower is better. All of these metrics are displayed as percentages.

```
   |_____________________MS_FT_____________________|______________________FPP______________________|_____________________AM_ZN_____________________|
   |  PPV  |  FDR  |  TPR  |  FPR  |  ACC  |  AGE  |  PPV  |  FDR  |  TPR  |  FPR  |  ACC  |  AGE  |  PPV  |  FDR  |  TPR  |  FPR  |  ACC  |  AGE  |
---|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
M  | 99.23 |  0.77 | 97.15 |  0.75 | 98.20 | 14.22 | 98.63 |  1.37 | 97.10 |  1.35 | 97.88 | 18.06 | 99.15 |  0.85 | 93.05 |  0.80 | 96.12 | 17.74 |
F  | 97.21 |  2.79 | 99.25 |  2.85 | 98.20 | 17.03 | 97.14 |  2.86 | 98.65 |  2.90 | 97.88 | 19.69 | 93.45 |  6.55 | 99.20 |  6.95 | 96.12 | 23.00 |
L  | 98.65 |  1.35 | 98.65 |  1.35 | 98.60 | 16.09 | 98.65 |  1.35 | 98.65 |  1.35 | 98.60 | 16.78 | 96.35 |  3.65 | 96.35 |  0.36 | 96.30 | 21.69 |
LM | 98.99 |  1.01 | 98.30 |  0.50 | 98.60 | 13.26 | 98.99 |  1.01 | 98.30 |  0.50 | 98.60 | 17.16 | 99.15 |  0.85 | 93.50 |  0.80 | 98.60 | 16.77 |
LF | 98.31 |  1.69 | 98.90 |  0.85 | 98.60 | 18.92 | 98.31 |  1.69 | 98.90 |  0.85 | 98.60 | 16.41 | 93.84 |  6.16 | 99.10 |  6.50 | 96.30 | 26.62 |
D  | 97.75 |  2.25 | 97.75 |  2.50 | 97.80 | 15.25 | 97.10 |  2.90 | 97.10 |  2.90 | 97.15 | 21.22 | 95.90 |  4.10 | 95.90 |  4.10 | 95.95 | 19.17 |
DM | 99.48 |  0.52 | 96.00 |  0.50 | 97.80 | 15.37 | 98.25 |  1.75 | 95.90 |  1.70 | 97.15 | 19.14 | 93.06 |  6.94 | 92.60 |  0.80 | 95.95 | 18.90 |
DF | 96.14 |  3.86 | 99.50 |  4.00 | 97.80 | 15.15 | 96.00 |  4.00 | 98.30 |  4.10 | 97.15 | 22.97 | 93.06 |  6.94 | 99.20 |  7.41 | 95.95 | 19.39 |
```

# Conclusion

I will preface this section by highlighting an intriguing anomaly - 9 subjects failed analysis altogether, all of whom were dark-skinned males. Azure failed as such 8 times and Rekognition did so once, while Face++ successfully detected every subject (even if it failed to analyze them correctly). This data is recorded at `failures.json`, reference [3].

[3]: https://github.com/Nytelife26/transparency/blob/main/works/personal/2021/nrp

The data collected by the study determines that, for all three technologies, error rates are highest for females of darker skin tones. In terms of age accuracy, Azure was least accurate on females, Face++ was least accurate on people of darker skin tones (darker-skinned females in particular), and Rekognition was by far least accurate for females of lighter skin tones with an average $8.85\%$ less accuracy in comparison with males of lighter skin tones.

All three technologies had the worst accuracy scores for people of darker skin tones, and the highest false positive rates for females of higher skin tones with an average of $7.76\%$.

In conclusion, people of darker skin tones and females of any skin tone have the worst detection rates from the AI technologies tested, leaving males of lighter skin tones at the forefront of their acuity. While there has been a significant improvement since the Gender Shades project, with a reduced error rate for dark-skinned females from $30.97\%$ to $4.93\%$, there is still evidence of the prior discriminatory practice in facial recognition technologies.

# Conflicts of Interest

None applicable.

# References

1. [A. Najibi for Harvard's Science in the News, 2020](https://sitn.hms.harvard.edu/flash/2020/racial-discrimination-in-face-recognition-technology)
2. [Gender Shades Project, 2018](http://proceedings.mlr.press/v81/buolamwini18a/buolamwini18a.pdf)
3. [T. Russell, 2021](https://github.com/Nytelife26/transparency/blob/main/works/personal/2021/nrp)
4. [IEEE, 2017](https://susanqq.github.io/UTKFace/)
5. [Z. Liu, P. Luo, X. Wang, & X. Tang, 2015](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)
6. [T. Russell, 2021](https://github.com/Nytelife26/transparency/blob/main/works/personal/2021/nrp/final.tar.gz)

# Acknowledgements

I give my utmost gratitude to the directors behind the Nuffield Research Placements scheme, and my supervisor, Professor M. Elshaw of Coventry University. In addition, I credit the Gender Shades project for inspiring a large part of the methodology behind this study.